##SETUP

In [9]:
import os
import sys
import time
import pandas as pd
import numpy as np
# Android ADB
from scrcpy import Client, const

# Image processing
from PIL import Image
import cv2
import pytesseract
# Notebook
from IPython.display import display,clear_output
from tqdm.notebook import trange, tqdm
# internal
sys.path.append('./src')
import bot_core
import bot_perception

import importlib
importlib.reload(bot_core)
importlib.reload(bot_perception)

bot = bot_core.Bot()
# Evaluate grid to update OCR images
names=bot.scan_grid()
grid_df=bot_perception.grid_status(names)
display(grid_df)
df_split,unit_series, df_groups, group_keys=bot_core.grid_meta_info(grid_df)
merge_series = unit_series.copy()


,grid_pos,unit,probability,rank,rank_prob,Age
0,"[0, 0]",empty.png,999.0,0,1.000000,0.0
1,"[0, 1]",zealot.png,495.0,1,0.999991,0.0
2,"[0, 2]",dryad.png,223.0,2,1.000000,0.0
3,"[0, 3]",empty.png,700.0,1,0.999933,0.0
4,"[0, 4]",dryad.png,349.0,1,0.999998,0.0
5,"[1, 0]",empty.png,700.0,0,0.999592,0.0
6,"[1, 1]",harlequin.png,200.0,1,1.000000,0.0
7,"[1, 2]",bruser.png,587.0,1,1.000000,0.0
8,"[1, 3]",zealot.png,298.0,1,1.000000,0.0
9,"[1, 4]",empty.png,999.0,0,1.000000,0.0


In [13]:
# Check if alive
bot.client.control.text("test")


b'\x01\x00\x00\x00\x04test'

In [10]:
bot.getScreen()

In [39]:
# Restart game
bot.restart_RR()

## Performance Optimizations

In [10]:
# profile bot.try_merge
import cProfile
cProfile.run('bot.try_merge(prev_grid=grid_df)','profile.txt')
# sort by time
import pstats
p = pstats.Stats('profile.txt')
p.sort_stats('time').print_stats(5)

unit        rank
dryad.png   1       4
zealot.png  1       3
Name: unit, dtype: int64
unit       rank
dryad.png  1       4
Name: unit, dtype: int64 unit        rank
zealot.png  1       3
Name: unit, dtype: int64 unit        rank
dryad.png   1       4
zealot.png  1       3
Name: unit, dtype: int64
Merged special!
Sat Jul 16 14:33:48 2022    profile.txt

         65636 function calls (65002 primitive calls) in 2.522 seconds

   Ordered by: internal time
   List reduced from 1023 to 5 due to restriction <5>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    1.827    0.913    1.827    0.913 {built-in method nt.system}
       12    0.393    0.033    0.393    0.033 {built-in method time.sleep}
      102    0.127    0.001    0.127    0.001 {imread}
      142    0.086    0.001    0.086    0.001 {method 'detectAndCompute' of 'cv2.Feature2D' objects}
       15    0.013    0.001    0.013    0.001 {imwrite}




## Merge Logic

In [ ]:
## From Try_merge
# Remove all high level crystals and zealots
merge_series = bot_core.adv_filter_keys(merge_series,[[3,4,5],['zealot.png','crystal.png']],remove=True)
# Select stuff to merge
# Find highest chemist rank
merge_chemist = bot_core.adv_filter_keys(unit_series,'chemist.png',remove=False)
if not merge_chemist.empty:
    max_chemist = merge_chemist.index.max()
    # Remove 1 count of highest rank chemist
    merge_series[merge_series.index == max_chemist] = merge_series[merge_series.index == max_chemist] - 1
# Select stuff to merge
merge_series = merge_series[merge_series>=2] # At least 2 units ## ADD MIME to every count, use sample rank and check if mime exist

In [ ]:
def merge_special_unit(df_split,merge_series,merge_harley=False):
    # Merge harley if exists
    harley_merge, normal_unit = [bot_core.adv_filter_keys(merge_series,'harlequin.png',remove=remove) for remove in [False,True]]
    if not harley_merge.empty:
        # Get corresponding dataframes
        harley_merge, normal_df = [df_split.get_group(unit.index[0]).sample() for unit in [harley_merge, normal_unit]]
        merge_df=pd.concat([harley_merge, normal_df])
        # Do Harley merge
        unit_chosen=merge_df['position'].tolist()
        bot.swipe(*unit_chosen)
        if merge_harley:
            print('Double merged Harley!')
            time.sleep(0.5)
            bot.swipe(*unit_chosen)
        return merge_df
    # Get other special merge unit
    special_unit, normal_unit=[bot_core.adv_filter_keys(merge_series,[['dryad.png','mime.png','scrapper.png']],remove=remove) for remove in [False,True]] # scrapper support not tested
    # Get corresponding dataframes
    print(special_unit, normal_unit,merge_series)
    special_df, normal_df = [df_split.get_group(unit.index[0]).sample() for unit in [special_unit, normal_unit]]
    merge_df=pd.concat([special_df, normal_df])
    # Merge 'em
    unit_chosen=merge_df['position'].tolist()
    bot.swipe(*unit_chosen)
    time.sleep(0.2)
    print('Merged special!')
    return merge_df
def special_merge(merge_series):
    merge_df = None
    # Try to rank up dryads
    dryads_series=bot_core.adv_filter_keys(merge_series,'dryad.png')
    if not dryads_series.empty:
        dryads_rank = dryads_series.index.get_level_values('rank')
        for rank in dryads_rank:
            merge_series_dryad=bot_core.adv_filter_keys(merge_series,[rank,['harlequin.png','dryad.png']])
            merge_series_zealot=bot_core.adv_filter_keys(merge_series,[rank,['zealot.png','dryad.png']])
            if len(merge_series_dryad.index)==2:
                merge_df = merge_special_unit(df_split,merge_series_dryad)
                break
            if len(merge_series_zealot.index)==2:
                print(merge_series_zealot)
                merge_df = merge_special_unit(df_split,merge_series_zealot)
                break
    return merge_df

In [238]:
grid_df =None
battle_bar=trange(100)
for i in battle_bar:
    # live test
    merge_df = None
    names=bot.scan_grid(new=True)
    grid_df=bot_perception.grid_status(names)
    df_split,unit_series, df_groups, group_keys=bot_core.grid_meta_info(grid_df)
    merge_series = unit_series.copy()
    bot.special_merge(df_split,merge_series)
    print(df_groups)
    print(merge_df)

  0%|          | 0/100 [00:00<?, ?it/s]

unit
cauldron.png     4
chemist.png      1
empty.png        4
harlequin.png    1
zealot.png       5
Name: unit, dtype: int64
None
unit
cauldron.png    3
chemist.png     2
empty.png       5
zealot.png      5
Name: unit, dtype: int64
None
unit
cauldron.png    4
chemist.png     1
empty.png       5
zealot.png      5
Name: unit, dtype: int64
None
unit
cauldron.png    3
chemist.png     2
empty.png       5
zealot.png      5
Name: unit, dtype: int64
None
unit
cauldron.png    3
chemist.png     2
empty.png       5
zealot.png      5
Name: unit, dtype: int64
None
unit
cauldron.png    2
chemist.png     3
empty.png       5
zealot.png      5
Name: unit, dtype: int64
None
unit
cauldron.png     3
chemist.png      1
empty.png        5
harlequin.png    1
zealot.png       5
Name: unit, dtype: int64
None
unit
cauldron.png    3
chemist.png     2
empty.png       5
zealot.png      5
Name: unit, dtype: int64
None
unit
cauldron.png     2
chemist.png      1
empty.png        5
harlequin.png    2
zealot.png       

## Watch ads

In [27]:
store_state = bot.get_store_state()
store_state

'spin_only'

In [25]:
'spin_only' in ['nothing','spin_only']

True

In [21]:
x,y = [140,1412]
img_rgb = cv2.imread(bot.screenshotName)
store_rgb = img_rgb[y:y + 1, x:x + 1]
store_rgb = store_rgb[0][0]
store_rgb

array([ 80, 153, 193], dtype=uint8)

In [18]:
bot.refresh_shop()

refreshed!


'refresh'

In [ ]:
# Navigate and locate store refresh button from battle screen
def find_store_refresh(self):
    self.click_button((100,1500)) # Click store button
    [self.swipe([0,0],[2,0]) for i in range(20)] # swipe to top
    self.swipe([2,0],[0,0]) # Swipe down once
    time.sleep(1)
    self.click(30,150) # stop scroll
    avail_buttons = self.get_current_icons(available=True)
    if (avail_buttons == 'refresh_button.png').any(axis=None):
        pos = bot_core.get_button_pos(avail_buttons,'refresh_button.png')
        return pos
# Refresh items in shop when available 
def refresh_shop(self):
    store_state = self.get_store_state()
    if store_state == 'nothing':
        return store_state
    elif store_state == 'new_offer':
        self.click_button((100,1500)) # Click store button
        self.click_button((500,1500)) # Click battle button
        self.click_button((100,1500)) # Click store button
    elif store_state == 'refresh':    
        pos = self.find_store_refresh()
        if not pos is None:
            self.click_button(pos)
            print('refreshed!')
    elif store_state == 'new_store':    
        pos = self.find_store_refresh()
        if not pos is None:
            # Buy first and last item (possible legendary) before refresh
            self.click_button(pos-[300,820]) # Click first (free) item
            self.click(400,1150) # buy
            self.click(30,150) # remove pop-up
            self.click_button(pos+[400,-400]) # Click first (free) item
            self.click(400,1150) # buy
            print('Bought!')
    return store_state

refresh_shop(bot)

In [186]:
avail_buttons = bot.get_current_icons(available=True)
pos = bot_core.get_button_pos(avail_buttons,'refresh_button.png')


array([ 81, 306], dtype=int64)

In [200]:
avail_buttons = bot.get_current_icons(available=True)
display(avail_buttons)

,icon,available,"pos [X,Y]"
0,battle_icon.png,True,"(359, 1419)"
1,pvp_button.png,True,"(140, 1259)"


## Get grid meta

In [4]:
names=bot.scan_grid()
grid_df=bot_perception.grid_status(names)
grid_df=bot_perception.grid_status(names,grid_df)
display(grid_df)
prev_grid= grid_df.copy()

,grid_id,unit,probability,rank,rank_error,position,Age
0,icon_0,crystal.png,185.0,2,605.0,"[0, 0]",1.0
1,icon_1,crystal.png,385.0,3,178.5,"[0, 1]",1.0
2,icon_2,dryad.png,291.0,2,585.0,"[0, 2]",1.0
3,icon_3,monkey.png,556.0,2,610.0,"[0, 3]",1.0
4,icon_4,hunter.png,249.0,3,213.0,"[0, 4]",1.0
5,icon_5,monkey.png,324.0,1,1217.0,"[1, 0]",1.0
6,icon_6,dryad.png,664.0,2,275.0,"[1, 1]",1.0
7,icon_7,empty.png,999.0,0,0.0,"[1, 2]",1.0
8,icon_8,hunter.png,670.0,1,1489.0,"[1, 3]",1.0
9,icon_9,crystal.png,270.0,2,605.0,"[1, 4]",1.0


In [278]:
names=bot.scan_grid()
grid_df=bot_perception.grid_status(names)
df_split,unit_series, df_groups, group_keys=bot_core.grid_meta_info(grid_df)
display(unit_series)
#merge_series = unit_series[unit_series>=2]


unit         rank
empty.png    0       8
crystal.png  2       2
chemist.png  2       1
             3       1
hunter.png   3       1
             5       1
zealot.png   4       1
Name: unit, dtype: int64

In [250]:
#('empty.png',0) in group_keys
df_groups['empty.png']<=2

False

## Advanced key filtering

In [252]:
merge_series = unit_series

bot_core.adv_filter_keys(merge_series,[['chemist.png','monkey.png']])

unit         rank
chemist.png  3       1
monkey.png   4       1
Name: unit, dtype: int64

In [437]:
# Setup test
multi_index=pd.MultiIndex.from_tuples([
            ( 'monkey.png', 1),
            ( 'monkey.png', 4),
            ('crystal.png', 2),
            ('crystal.png', 4),
            ('zealot.png', 2),
            ('zealot.png', 4),],
           names=['unit', 'rank'])
unit_series_test=pd.Series([2,2,2,2,2,2],index=multi_index)
# Pre-condition
merge_series = unit_series_test.copy()
merge_series = bot_core.adv_filter_keys(merge_series,'empty.png',remove=True)
merge_chemist = bot_core.adv_filter_keys(unit_series,'chemist.png',remove=False)
if not merge_chemist.empty:
    max_chemist = merge_chemist.index.max()
    # Remove 1 count of highest rank chemist
    merge_series[merge_series.index == max_chemist] = merge_series[merge_series.index == max_chemist] - 1
## Select stuff to merge
merge_series = merge_series[merge_series>=2] # At least 2 units ## ADD MIME to every count, use sample rank and check if mime exist
# Remain
#merge_series = bot_core.adv_filter_keys(merge_series,[[3,4,5],'crystal.png'],remove=True)
bot_core.adv_filter_keys(merge_series,[[4,5],['zealot.png','crystal.png']],remove=True)

#bot_core.adv_filter_keys(merge_series,[['zealot.png','crystal.png']],remove=True)


[4, 5] 4
[4, 5] 5
123
['zealot.png', 'crystal.png'] zealot.png
['zealot.png', 'crystal.png'] crystal.png
123


unit         rank
monkey.png   1       2
             4       2
crystal.png  2       2
zealot.png   2       2
dtype: int64

In [427]:
# Returns all elements which match tokens value with multiple levels
# Either provide nested list of list, simple list or unit type/unit rank and if remove or not
# Criteria example:  [[3,4,5],['zealot.png','crystal.png']]
def adv_filter_keys(unit_series,tokens,remove=False):
    if unit_series.empty:
        return pd.Series(dtype=object)
    if not isinstance(tokens, list): # Make token a list if not already
        tokens = [tokens]
    # Add detection of dimension in input tokens
    merge_series= unit_series.copy()
    for level in tokens:
        merge_series_temp= merge_series.copy()
        if not isinstance(level, list): # Make token a list if not already
            level = [level]
        series= []
        for token in level:
            print(level,token)
            # check if given token is int, assume unit rank filter
            if isinstance(token,int):
                exists = merge_series.index.get_level_values('rank').isin([token]).any()
                if exists:
                    series.append(merge_series.xs(token, level='rank',drop_level=False))
                else: continue # skip if nothing matches criteria
            elif isinstance(token,str):
                if token in merge_series: 
                    series.append(merge_series.xs(token, level='unit',drop_level=False))
                else: continue
        # Every iteration
        # If any matches are found
        if not len(series)==0:
            merge_series = pd.concat(series)
            # Select matches in previous matches 
            merge_series = merge_series_temp[merge_series_temp.index.isin(merge_series.index)]
         # return empty list if empty and nothing matches criteria
        elif not remove:
            return pd.Series(dtype=object)
        # if removing matches from initial series and no matches are found, do nothing this loop, keep list same
        else: 
            continue
    # LOOP DONE
    if remove:
        # Remove all matches found from original series
        merge_series = unit_series[~unit_series.index.isin(merge_series.index)]
    # Return matches found
    return merge_series



        # Otherwise Do next loop with unchanged merge series otherwise
    # Return result of all criterias
    print(555)
    return merge_series
#unit_series

In [331]:

merge_series = unit_series.copy()
merge_series = adv_filter_keys(merge_series,'empty.png',remove=True)
merge_chemist = adv_filter_keys(unit_series,'chemist.png',remove=False)
if not merge_chemist.empty:
    max_chemist = merge_chemist.index.max()
    # Remove 1 count of highest rank chemist
    merge_series[merge_series.index == max_chemist] = merge_series[merge_series.index == max_chemist] - 1
    display(merge_series)
## Select stuff to merge
merge_series = merge_series[merge_series>=2] # At least 2 units ## ADD MIME to every count, use sample rank and check if mime exist
merge_prio = adv_filter_keys(merge_series,[['chemist.png','monkey.png']],remove=True)
#if not merge_prio.empty:
#    print('merge!')
display(merge_prio)

unit         rank
crystal.png  2       2
chemist.png  2       1
             3       0
hunter.png   3       1
             5       1
zealot.png   4       1
Name: unit, dtype: int64

unit         rank
crystal.png  2       2
Name: unit, dtype: int64

In [311]:
merge_series

unit         rank
crystal.png  2       2
Name: unit, dtype: int64

In [287]:
unit_series

unit         rank
empty.png    0       8
crystal.png  2       2
chemist.png  2       1
             3       1
hunter.png   3       1
             5       1
zealot.png   4       1
Name: unit, dtype: int64

In [205]:
merge_series = unit_series_test
adv_filter_keys(merge_series,[[3,4,5],['zealot.png','crystal.png']],remove=True)


unit         rank
monkey.png   1       2
crystal.png  2       2
             4       2
zealot.png   2       2
             4       2
dtype: int64

In [ ]:
bot_core.adv_filter_keys(merge_series,[
    [3,4,5],
    ['crystal.png','zealot.png']
    ],remove=True)

In [97]:
# Find highest chemist rank
merge_series = adv_filter_keys(unit_series,'chemist.png',remove=False)
max_chemist = merge_series.index.max()
# Remove 1 count of highest rank chemist
merge_series[merge_series.index == max_chemist] = merge_series[merge_series.index == max_chemist] - 1
merge_series

unit         rank
chemist.png  4       0
Name: unit, dtype: int64

In [34]:
# Remove tuple tokens from unit series
def remove_keys(unit_series,tokens = [('empty.png', 0)]):
    return unit_series[~unit_series.index.isin([tokens])]
merge_series 

array([False,  True, False,  True, False, False, False, False, False,
       False, False])

In [37]:
merge_series[~merge_series.index.isin([('crystal.png', 1),('crystal.png', 2)])]

unit         rank
dryad.png    2       3
empty.png    0       2
crystal.png  3       1
dryad.png    1       1
hunter.png   1       1
             3       1
monkey.png   1       1
             2       1
             3       1
Name: unit, dtype: int64